## Pipeline de um processo de Deep Learning implementado em PyTorch:

    1. Preparar os Dados
    2. Definir o Modelo
    3. Treinar o Modelo
    4. Avaliar o Modelo
    5. Usar o Modelo


## **MLP para classificação multiclasse**
**Dataset Iris (flores)**
- Dataset de imagens para previsão da espécie da flor dadas as medidas das flores
- 3 classes com 50 instâncias cada
- Classes: Iris Setosa, Iris Versicolour e Iris Virginica
- 5 atributos: 4 para dimensões numéricas e o 5º a classe da flor

In [53]:
import torch
print(torch.__version__)

1.10.2


## Imports

In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
PATH = 'iris.csv'

device = torch.device("cpu")

EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.01

## 1. Preparar os Dados

In [56]:
class CSVDataset(Dataset):
    def __init__(self, path):
        df = pd.read_csv(path, header=None)
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        self.X = self.X.astype('float32')
        self.y = LabelEncoder().fit_transform(self.y)
    
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])


    
def prepare_data(path):
    dataset = CSVDataset(path)
    train, test = dataset.get_splits()
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    train_dl_all = DataLoader(train, batch_size=len(train), shuffle=False)
    test_dl_all = DataLoader(test, batch_size=len(test), shuffle=False)
    return train_dl, test_dl, train_dl_all, test_dl_all

train_dl, test_dl,  train_dl_all, test_dl_all = prepare_data(PATH)

## 2. Definir o Modelo

In [57]:
from torchinfo import summary

class MLP(Module):
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        self.hidden3 = Linear(8,3)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax()
 
    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        return X

model = MLP(4)
print(summary(model, input_size=(BATCH_SIZE, 4), verbose=0))
model.to(device)

Layer (type:depth-idx)                   Output Shape              Param #
MLP                                      --                        --
├─Linear: 1-1                            [32, 10]                  50
├─ReLU: 1-2                              [32, 10]                  --
├─Linear: 1-3                            [32, 8]                   88
├─ReLU: 1-4                              [32, 8]                   --
├─Linear: 1-5                            [32, 3]                   27
├─Softmax: 1-6                           [32, 3]                   --
Total params: 165
Trainable params: 165
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01


/tmp/ipykernel_14536/935373995.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = self.act3(X)


MLP(
  (hidden1): Linear(in_features=4, out_features=10, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=10, out_features=8, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=8, out_features=3, bias=True)
  (act3): Softmax(dim=None)
)

## 3. Treinar o Modelo

In [58]:
def train_model(train_dl, model):
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.5)
    for epoch in range(EPOCHS):
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            yprev = model(inputs)
            loss = criterion(yprev, targets)
            loss.backward()
            optimizer.step()

train_model(train_dl, model)

/tmp/ipykernel_14536/935373995.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = self.act3(X)


## 4. Avaliar o Modelo

In [59]:
def evaluate_model(test_dl, model):
    predictions = list()
    actual_values = list()
    for i, (inputs, labels) in enumerate(test_dl):
        yprev = model(inputs)
        yprev = yprev.detach().numpy()
        actual = labels.numpy()
        yprev = np.argmax(yprev, axis=1)
        actual = actual.reshape((len(actual), 1))
        yprev = yprev.reshape((len(yprev), 1))
        predictions.append(yprev)
        actual_values.append(actual)
        break
    predictions, actual_values = np.vstack(predictions), np.vstack(actual_values)
    return predictions, actual_values
 
def display_confusion_matrix(cm):
    plt.figure(figsize = (16,8))
    sns.heatmap(cm, annot=True, xticklabels=['Iris-setosa', 'Iris-versicolor','Iris-virginica'], yticklabels=['Iris-setosa', 'Iris-versicolor','Iris-virginica'], annot_kws={"size": 12}, fmt='g', linewidth=.5)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show() 
    
predictions, actual_values = evaluate_model(test_dl, model)

acertou=0
falhou = 0
for r,p in zip(actual_values, predictions):
    print(f'real:{r} previsão:{p}') 
    if r==p: acertou+=1  
    else: falhou+=1

acc = accuracy_score(actual_values, predictions)
print(f'Accuracy: {acc:0.3f}\n')
print(f'acertou:{acertou} falhou:{falhou}')

real:[0] previsão:[0]
real:[0] previsão:[0]
real:[2] previsão:[2]
real:[1] previsão:[2]
real:[2] previsão:[2]
real:[1] previsão:[2]
real:[2] previsão:[2]
real:[0] previsão:[0]
real:[1] previsão:[2]
real:[1] previsão:[2]
real:[2] previsão:[2]
real:[2] previsão:[2]
real:[2] previsão:[2]
real:[1] previsão:[2]
real:[2] previsão:[2]
real:[2] previsão:[2]
real:[0] previsão:[0]
real:[0] previsão:[0]
real:[0] previsão:[0]
real:[0] previsão:[0]
real:[2] previsão:[2]
real:[1] previsão:[2]
real:[1] previsão:[2]
real:[0] previsão:[0]
real:[1] previsão:[2]
real:[0] previsão:[0]
real:[2] previsão:[2]
real:[1] previsão:[2]
real:[1] previsão:[2]
real:[1] previsão:[2]
real:[0] previsão:[0]
real:[2] previsão:[2]
real:[1] previsão:[2]
real:[1] previsão:[2]
real:[0] previsão:[0]
real:[0] previsão:[0]
real:[2] previsão:[2]
real:[0] previsão:[0]
real:[0] previsão:[0]
real:[2] previsão:[2]
real:[1] previsão:[2]
real:[1] previsão:[2]
real:[0] previsão:[0]
real:[1] previsão:[2]
real:[0] previsão:[0]
real:[1] p

/tmp/ipykernel_14536/935373995.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = self.act3(X)


## 5. Usar o Modelo

In [60]:
def predict(row, model):
    row = Tensor([row])
    yprev = model(row)
    yprev = yprev.detach().numpy()
    return yprev

row = [5.1,3.5,1.4,0.2]
yprev = predict(row, model)
print('Predicted: %s (class=%d)' % (yprev, np.argmax(yprev)))

Predicted: [[0.46083403 0.24265315 0.29651275]] (class=0)


/tmp/ipykernel_14536/935373995.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = self.act3(X)
